# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/openbmb/MiniCPM-V-4_5',
 '/tencent/Hunyuan-MT-7B',
 '/meituan-longcat/LongCat-Flash-Chat',
 '/Qwen/Qwen-Image-Edit',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/apple/fastvlm-webgpu',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/multimodalart/Qwen-Image-Edit-Fast',
 '/spaces/bytedance-research/USO',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/openai/healthbench',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/facebook/recycling_the_web',
 '/datasets/syncora/synthetic-healthcare-admissions',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micro

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discord page', 'url': 'https://huggingface.co/join/discord'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborate

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\n1 day ago\n•\n134k\n•\n1.25k\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 2 hours ago\n•\n13.9k\n•\n827\ntencent/Hunyuan-MT-7B\nUpdated\nabout 1 hour ago\n•\n487\n•\n354\nmeituan-longcat/LongCat-Flash-Chat\nUpdated\n2 days ago\n•\n5.24k\n•\n340\nQwen/Qwen-Image-Edit\nUpdated\n8 days ago\n•\n84.5k\n•\n1.62k\nBrowse 1M+ models\nSpaces\nRunning\n12.6k\n12.6k\nDeep

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
Hugging Face is an innovative platform at the forefront of artificial intelligence (AI) and machine learning (ML) that serves as a collaborative community for creators and developers. Our mission is to build the future of AI together, fostering a supportive ecosystem for sharing knowledge, models, datasets, and applications.

### Our Offerings
- **Models** - Access a vast library of over 1 million models across various AI modalities including text, image, audio, video, and even 3D.
- **Datasets** - Explore a collection of 250,000+ datasets tailored for diverse ML tasks, enabling researchers and developers alike to enhance their projects.
- **Spaces** - A dedicated space for community-driven applications, which currently hosts over 400,000 different usage applications for users to experience and improve upon. 

### Enterprise Solutions
Hugging Face provides specialized enterprise solutions designed to cater to organizational needs, including:
- **Compute**: Scalable and optimized inference endpoints available starting at $0.60/hour.
- **Team Support**: Advanced features for teams including Single Sign-On, priority support, resource groups, and custom audit logs, starting at $20/user/month.

---

## Our Community
With more than 50,000 organizations using Hugging Face—including major names like Microsoft, Google, and Amazon—our platform is a trusted resource for businesses and enthusiasts. We pride ourselves on creating a welcoming community where users can collaborate and learn from one another.

## Company Culture
At Hugging Face, we believe in an open-source philosophy and community-driven innovation. Our culture thrives on collaboration, inclusivity, and continuous learning. We encourage team members to share their expertise and insights, fostering a dynamic environment that celebrates creativity and problem-solving.

---

## Careers
Hugging Face welcomes passionate individuals who are eager to contribute to the AI community. We are currently looking for talented professionals who share our vision and mission. 

### Why Join Us?
- Contribute to cutting-edge AI technologies.
- Work within a vibrant and collaborative team.
- Opportunities for professional growth and impact.

Explore current job openings on our [Careers Page](https://huggingface.co/jobs) and join us in building the future of AI.

---

### Connect with Us
Stay updated with our latest developments, news, and community contributions through our social media channels:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

Join the AI revolution with Hugging Face and help shape tomorrow's technologies today!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Company Brochure

## About Us
Hugging Face is a pioneering AI community dedicated to building the future through collaboration. We create a platform where machine learning enthusiasts, researchers, and developers come together to share models, datasets, and applications. Our vision is to facilitate the discovery and development of state-of-the-art AI technologies for various modalities, including text, image, video, and audio.

## Our Offerings
- **Models**: Access over 1 million pre-trained models to streamline your machine learning projects.
- **Datasets**: Browse through a vast collection of over 250k datasets tailored for multiple ML tasks.
- **Spaces**: Engage with over 400k applications and run different AI models effortlessly.
- **Enterprise Solutions**: Customizable solutions with enterprise-grade security and dedicated support—tailored for teams and companies.

## Our Community
At Hugging Face, we believe in the power of collaboration and shared knowledge. Our platform is utilized by more than 50,000 organizations, including renowned companies such as Google, Microsoft, Amazon, and Meta. Together, we are building the foundation for ML tooling, with a strong commitment to open source.

### Notable Collaborations
- **AI at Meta**: 2.22k models and over 7.55k followers.
- **Amazon**: Contributing 20 models and engaging with 3.39k followers.
- **Google**: With 1.04k models, the company has a follower base of 27.2k.

## Company Culture
Our culture at Hugging Face thrives on innovation, diversity, and inclusion. We prioritize creating an open and welcoming environment that allows our team members to thrive creatively. Our team is composed of passionate individuals who are driven by a common goal—to advance AI technology for everyone.

## Careers at Hugging Face
We are constantly seeking talented individuals to join our dynamic team. Whether you are a developer, researcher, or designer, there are various roles available that cater to different interests and expertise. At Hugging Face, you will be empowered to contribute to groundbreaking projects that shape the future of AI.

### Join Us
If you're excited about AI and want to make an impact in a vibrant community, we invite you to explore our current job openings at [Hugging Face Careers](https://huggingface.co/jobs).

---

For more information, visit our website at [Hugging Face](https://huggingface.co) and follow us on our social media channels for updates and community insights!

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

### **About Us**
Hugging Face is a vibrant AI community dedicated to building the future of machine learning. Our collaborative platform serves as a hub where developers, researchers, and organizations converge to share and enhance their work on AI models, datasets, and applications. With over 1 million models and 250,000 datasets, Hugging Face enables users to explore the vast possibilities of machine learning in a friendly and supportive environment.

---

### **Our Offerings**
- **Models**: Access a diverse collection of state-of-the-art AI models across various modalities, including text, image, video, audio, and 3D.
- **Datasets**: Share and discover a multitude of datasets tailored for machine learning tasks.
- **Spaces**: Collaborate and host applications that leverage our cutting-edge technologies.
- **Enterprise Solutions**: Scale effectively with our paid compute and enterprise offerings that prioritize security and dedicated support.

---

### **Company Culture**
At Hugging Face, we believe in the power of community and open-source collaboration. Our culture fosters innovation through shared knowledge, encouraging members to build their portfolios and showcase their work. We pride ourselves on inclusivity, ensuring that everyone, regardless of their experience level, feels welcomed and empowered to contribute.

---

### **Our Customers**
More than 50,000 organizations trust Hugging Face, from startups to large enterprises. Our clients include:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Intel**
- **Grammarly**

We are proud to support a the dynamic work of AI professionals and researchers who are pushing the boundaries of what's possible with machine learning.

---

### **Careers at Hugging Face**
Join our mission to democratize AI! We're always on the lookout for talented individuals who are passionate about machine learning and AI technologies. Whether you're a seasoned professional or just starting your career, there are exciting opportunities at Hugging Face across various roles. Explore our **Jobs** page to find your place in our rapidly-growing team.

---

### **Get Involved**
Ready to join us in building the future of AI? 
- **Explore**: [AI Apps](#)
- **Browse Models**: [1M+ Models](#)
- **Join the Community**: [Sign Up](#) 

Together, we can shape the landscape of artificial intelligence for generations to come!

---

**Contact Us:**  
For sales inquiries or to know more about our enterprise solutions, please reach out to our support team or visit our [website](#).

**Social Media:**  
Stay updated with our latest news and offerings on [GitHub](#), [Twitter](#), [LinkedIn](#), and [Discord](#). 

---

**Hugging Face** – The AI community building the future.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>